In [37]:
#xay dung ung dung tim kiem anh
import cv2
import numpy as np
import os
import pywt

def wavelet_transform(image_path):
    # print(f"Reading image from {image_path}")
    image = cv2.imread(image_path)
    coeffs2 = pywt.dwt2(image, 'haar')
    LL, (LH, HL, HH) = coeffs2
    return LL

def wavelet_hash(LL, size=64):
    LL_small = cv2.resize(LL, (size, size), interpolation=cv2.INTER_AREA)
    mean = LL_small.mean()
    hash_bits = (LL_small > mean).astype(np.uint8)
    return hash_bits.flatten()

def hamming_distance(hash1, hash2):
    return np.sum(hash1 != hash2)

def build_database(image_folder):
    database = []
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        # print(f"Processing {img_name}...")
        LL = wavelet_transform(img_path)
        # print(f"LL shape for {img_name}: {LL.shape}")
        h = wavelet_hash(LL)
        database.append((img_name, h))
    return database

def search_image(query_img, database, top_k=5):
    query_LL = wavelet_transform(query_img)
    query_hash = wavelet_hash(query_LL)

    results = []
    for img_name, h in database:
        dist = hamming_distance(query_hash, h)
        similarity = 1 - dist / len(h)
        if similarity == 1:
            results.append((img_name, similarity))
    return results

database = build_database("dataset")

results = search_image("12.jpg", database)

print("KẾT QUẢ TÌM KIẾM:")
if not results:
    print("Không tìm thấy ảnh giống trong cơ sở dữ liệu.")
for img, score in results:
    kq = cv2.imread(os.path.join("dataset", img))
    plt.imshow(cv2.cvtColor(kq, cv2.COLOR_BGR2RGB))
    plt.title(f"Similarity: {score:.3f}")
    plt.show()
    print(f"{img} - Similarity: {score:.3f}")


KẾT QUẢ TÌM KIẾM:
Không tìm thấy ảnh giống trong cơ sở dữ liệu.
